In [11]:
# Install langchain, rank-bm25, PyPDF2, unstructured, chromadb
!pip install langchain rank-bm25 PyPDF2 chromadb

# Install extra dependencies for handling PDFs
!pip install unstructured['pdf'] 

# Install poppler-utils for PDF processing
!apt-get install -y poppler-utils

# Install tesseract-ocr and its dependencies
!apt-get install -y tesseract-ocr libtesseract-dev

# Install pytesseract for OCR capabilities
!pip install pytesseract


  Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
  Using cached unstructured-0.13.3-py3-none-any.whl (1.9 MB)
  Using cached emoji-2.11.1-py2.py3-none-any.whl (433 kB)
  Using cached rapidfuzz-3.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)
  Using cached lxml-5.2.1-cp310-cp310-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached python_iso639-2024.2.7-py3-none-any.whl (274 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached unstructured_client-0.22.0-py3-none-any.whl (28 kB)
  Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 3.2 MB/s eta 0:00:00m eta 0

In [17]:
from langchain.document_loaders import PyPDFLoader,UnstructuredPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

from langchain_community.llms import HuggingFaceHub

from langchain.retrievers import BM25Retriever, EnsembleRetriever
import os
from dotenv import load_dotenv

In [13]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()
hf_api_key = os.getenv("hf_key")
print("Hugging Face API Key:", hf_api_key)

Hugging Face API Key: ########################################

In [14]:
file_path = "./data/yolo.pdf"
data_file = UnstructuredPDFLoader(file_path)
docs = data_file.load()

[nltk_data] Downloading package punkt to /home/diwas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/diwas/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [15]:
# Create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)


In [18]:
# Create embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=hf_api_key,
    model_name="BAAI/bge-base-en-v1.5"
)

In [20]:
# Create vector store
vectorstore = Chroma.from_documents(chunks, embeddings)



In [21]:
# Create vector store retriever
vectorstore_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


In [22]:

# Create keyword retriever
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k = 3


In [23]:
# Create ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[vectorstore_retriever, keyword_retriever], weights=[0.5, 0.5]
)

In [24]:
# Create LLM
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    model_kwargs={"temperature": 0.3, "max_new_tokens": 1024},
    huggingfacehub_api_token=hf_api_key
)


/home/diwas/Documents/rag/rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/home/diwas/Documents/rag/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
template="""
<|system|>>
You are a helpful AI assistant that follows instruction extremely well .
Use the following  context to answer user question.
CONTEXT:{context}
</s>
<|user|>
{query}
</s>
<|assistant|>      
"""

In [27]:
# Create prompt template
prompt = ChatPromptTemplate.from_template(template)
output_parser=StrOutputParser()
chain=(
    {"context":ensemble_retriever,"query":RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)



In [28]:
print(chain.invoke("What is  YOLO"))


Human: 
<|system|>>
You are a helpful AI assistant that follows instruction extremely well .
Use the following  copntext to answer user question.

CONTEXT:[Document(page_content='including the time to fetch images from the camera and dis- play the detections.\n\nThe resulting system is interactive and engaging. While YOLO processes images individually, when attached to a webcam it functions like a tracking system, detecting ob- jects as they move around and change in appearance. A demo of the system and the source code can be found on our project website: http://pjreddie.com/yolo/.\n\ndirectly on full images. Unlike classiﬁer-based approaches, YOLO is trained on a loss function that directly corresponds to detection performance and the entire model is trained jointly.', metadata={'source': './data/yolo.pdf'}), Document(page_content='the test data can diverge from what the system has seen be- fore [3]. We compare YOLO to other detection systems on the Picasso Dataset [12] and the People

In [30]:
print(chain.invoke("What is Unified Detection"))

Human: 
<|system|>>
You are a helpful AI assistant that follows instruction extremely well .
Use the following  copntext to answer user question.

CONTEXT:[Document(page_content='3. Comparison to Other Detection Systems\n\nObject detection is a core problem in computer vision. Detection pipelines generally start by extracting a set of robust features from input images (Haar [25], SIFT [23], HOG [4], convolutional features [6]). Then, classiﬁers [36, 21, 13, 10] or localizers [1, 32] are used to identify objects in the feature space. These classiﬁers or localizers are run either in sliding window fashion over the whole im- age or on some subset of regions in the image [35, 15, 39]. We compare the YOLO detection system to several top de- tection frameworks, highlighting key similarities and differ- ences.', metadata={'source': './data/yolo.pdf'}), Document(page_content='2. Uniﬁed Detection\n\nWe unify the separate components of object detection into a single neural network. Our network u

In [32]:
print(chain.invoke("What are the Limitations of YOLO"))

Human: 
<|system|>>
You are a helpful AI assistant that follows instruction extremely well .
Use the following  copntext to answer user question.

CONTEXT:[Document(page_content='We also train a fast version of YOLO designed to push the boundaries of fast object detection. Fast YOLO uses a neural network with fewer convolutional layers (9 instead of 24) and fewer ﬁlters in those layers. Other than the size of the network, all training and testing parameters are the same between YOLO and Fast YOLO.\n\n(1)\n\n33\n\n4096\n\n7730} ×4} ×2Conv. Layers3x3x10243x3x1024\n\n5656256Conn. Layer\n\n33\n\nConn. LayerConv. Layer3x3x192Maxpool Layer2x2-s-2Conv. Layers1x1x1283x3x2561x1x2563x3x512Maxpool Layer2x2-s-2\n\n33\n\n2828512Conv. Layers1x1x2563x3x5121x1x5123x3x1024Maxpool Layer2x2-s-2\n\n33\n\n771024\n\n112112192\n\n77Conv. Layer7x7x64-s-2Maxpool Layer2x2-s-2\n\n771024\n\n4484483\n\n14141024Conv. Layers1x1x5123x3x10243x3x10243x3x1024-s-2\n\n33', metadata={'source': './data/yolo.pdf'}), Document